In [4]:
import numpy as np

A = np.array([[0,0,2,0,0,0,0,0],
              [0,0,3,0,0,0,0,0],
              [0,0,0,4,0,0,0,0],
              [0,0,0,0,5,1,8,0],
              [0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,2,0],
              [0,0,0,0,0,0,0,5],
              [0,0,0,0,0,0,0,0]
             ])

B = np.array([
        [0,0,0,0,4,0,0,7,2],
        [0,0,0,0,0,0,0,0,0],
        [0,1,0,0,0,0,3,0,0],
        [0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,2,0,0],
        [2,0,0,0,0,0,0,0,0],
        [0,0,0,5,0,0,0,0,0],
        [0,0,0,0,0,0,0,3,0]
    ])

A_in_w = np.sum(A, axis=0)
A_out_w = np.sum(A, axis=1)
B_in_w = np.sum(B, axis=0)
B_out_w = np.sum(B, axis=1)

A_in = np.sum(A>0, axis=0)
A_out = np.sum(A>0, axis=1)
B_in = np.sum(B>0, axis=0)
B_out = np.sum(B>0, axis=1)

# 0 => peripheral, 1 => bridge, 2 => center
A_role = np.array([0,0,1,2,0,0,0,0])
B_role = np.array([2,0,0,0,0,0,1,0,0])


def ego_connectivity(G):
    ego_conn = np.zeros((G.shape[0]))
    for i in range(G.shape[0]):
        ego_i = (G[i] | G.T[i])>0
        ego_i[i] = True
        ego_net = G[ego_i].T[ego_i]
        ego_conn[i] = np.sum(ego_net > 0)/float(ego_net.shape[0]**2)
    return ego_conn


A_ego_conn = ego_connectivity(A);
B_ego_conn = ego_connectivity(B);
print(A_ego_conn)
print(B_ego_conn)

A_feature = np.array([A_in, A_out, A_in_w, A_out_w, A_ego_conn])
print(A.shape)

[ 0.25        0.25        0.1875      0.2         0.25        0.33333333
  0.25        0.25      ]
[ 0.2         0.25        0.22222222  0.25        0.25        0.25        0.1875
  0.25        0.33333333]
(8, 8)


In [25]:
# produce graph A
import numpy as np
nodes = []
with open('facebook/0.edges') as f:
    for line in f:
        pair = line.strip().split()
        if int(pair[0]) == int(pair[1]):
            print("hey\n")
        nodes.append(int(pair[0]))
        nodes.append(int(pair[1]))
    
A_size = max(nodes)+1
A = np.zeros((A_size,A_size))

for i in range(0,len(nodes)-2,2):
    A[nodes[i]][nodes[i+1]]=1
    A[nodes[i+1]][nodes[i]]=1
    
A[0,:]=1
A[:,0]=1
A[0,0]=0

A_feats = np.zeros((A_size, 224))

with open('facebook/0.feat') as f:
    for line in f:
        feats = line.strip().split()
        A_feats[int(feats[0])] = feats[1:]
        
print(A_feats[3])

[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.
  0.  1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  0.  0.  0.  0.]


In [16]:
# produce permuted B with noise
perm = np.random.permutation(len(A))

P = np.identity(len(A))
P = P[perm]

B = P.dot(A).dot(P.T)
noise = np.random.choice([0, 1], size=(len(A),len(A)), p=[99.9/100, 0.1/100])
B = (B + noise)%2

print("noise edges: " + str(np.sum(np.abs(A-P.T.dot(B).dot(P))[:])))

noise edges: 116.0


In [2]:
from graph_tool.all import *



ImportError: dlopen(/usr/local/lib/python2.7/site-packages/graph_tool/libgraph_tool_core.so, 8): Symbol not found: __ZNK5boost16re_detail_10610031cpp_regex_traits_implementationIcE17transform_primaryEPKcS4_
  Referenced from: /usr/local/lib/python2.7/site-packages/graph_tool/libgraph_tool_core.so
  Expected in: /usr/local/opt/boost/lib/libboost_regex-mt.dylib
 in /usr/local/lib/python2.7/site-packages/graph_tool/libgraph_tool_core.so

In [3]:
import snap
import numpy as np
import pandas as pd

def getAttribute(filename):
    UGraph = snap.LoadEdgeList(snap.PUNGraph, filename, 0, 1)
    UGraph.Dump()

    attributes = pd.DataFrame(np.zeros(shape=(UGraph.GetNodes(), 11)), 
                              columns=['Id', 'Degree', 'DegreeCentrality', 'NodeBetweennessCentrality', 
                                       'ClosenessCentrality', 'FarnessCentrality', 'PageRank', 'HubsScore', 
                                       'AuthoritiesScore', 'NodeEccentricity', 'EigenvectorCentrality'])

    # Degree
    id = []
    degree = []
    OutDegV = snap.TIntPrV()
    snap.GetNodeOutDegV(UGraph, OutDegV)
    for item in OutDegV:
        id.append(item.GetVal1())
        degree.append(item.GetVal2())
    attributes['Id'] = id
    attributes['Degree'] = degree

    # Degree, Closeness, Farness Centrality, Node Eccentricity
    degCentr = []
    cloCentr = []
    farCentr = []
    nodeEcc = []
    for NI in UGraph.Nodes():
        degCentr.append(snap.GetDegreeCentr(UGraph, NI.GetId()))
        cloCentr.append(snap.GetClosenessCentr(UGraph, NI.GetId()))
        farCentr.append(snap.GetFarnessCentr(UGraph, NI.GetId()))
        nodeEcc.append(snap.GetNodeEcc(UGraph, NI.GetId(), False))
    attributes['DegreeCentrality'] = degCentr
    attributes['ClosenessCentrality'] = cloCentr
    attributes['FarnessCentrality'] = farCentr
    attributes['NodeEccentricity'] = nodeEcc

    # Betweenness Centrality
    betCentr = []
    Nodes = snap.TIntFltH()
    Edges = snap.TIntPrFltH()
    snap.GetBetweennessCentr(UGraph, Nodes, Edges, 1.0)
    for node in Nodes:
        betCentr.append(Nodes[node])
    attributes['NodeBetweennessCentrality'] = betCentr

    # PageRank
    pgRank = []
    PRankH = snap.TIntFltH()
    snap.GetPageRank(UGraph, PRankH)
    for item in PRankH:
        pgRank.append(PRankH[item])
    attributes['PageRank'] = pgRank

    # Hubs, Authorities score 
    hubs = []
    auth = []
    NIdHubH = snap.TIntFltH()
    NIdAuthH = snap.TIntFltH()
    snap.GetHits(UGraph, NIdHubH, NIdAuthH)
    for item in NIdHubH:
        hubs.append(NIdHubH[item])
    for item in NIdAuthH:
        auth.append(NIdAuthH[item])
    attributes['HubsScore'] = hubs
    attributes['AuthoritiesScore'] = auth

    # Eigenvector Centrality
    eigenCentr = []
    NIdEigenH = snap.TIntFltH()
    snap.GetEigenVectorCentr(UGraph, NIdEigenH)
    for item in NIdEigenH:
        eigenCentr.append(NIdEigenH[item])
    attributes['EigenvectorCentrality'] = eigenCentr

    return attributes

print getAttribute("facebook/0.edges")

      Id  Degree  DegreeCentrality  NodeBetweennessCentrality  \
0    236      36          0.108434                 200.150607   
1    186      43          0.129518                 302.943946   
2    122      62          0.186747                1250.238251   
3    285      46          0.138554                 752.212911   
4     24      15          0.045181                   5.902781   
5    346      26          0.078313                 255.675531   
6    271      72          0.216867                 822.781836   
7    304      54          0.162651                 744.431882   
8    176      13          0.039157                   8.938337   
9      9      56          0.168675                 541.363249   
10   130      15          0.045181                 548.399270   
11   329      29          0.087349                1754.716168   
12   204      21          0.063253                 893.096654   
13   213      38          0.114458                 983.175346   
14   252      64         